# Bokehのシンプルなグラフ描画

In [4]:
from bokeh.plotting import figure, output_notebook, output_file, show
from bokeh.models import GraphRenderer, Circle, MultiLine
from bokeh.models import Ellipse, GraphRenderer, StaticLayoutProvider
from bokeh.models import ColumnDataSource, Text, Plot

# プロットを作成
plot = figure(plot_width=400, plot_height=400)

# ネットワーク図のためのレンダラを作成
graph_renderer = GraphRenderer()

# (1) ネットワーク構造の定義
# ノードの定義
graph_renderer.node_renderer.data_source.data = {
    "index": ["A", "B", "C"],  # ノードID（必須）
    "my_color": ["gray", "red", "blue"],  # 任意の属性（スタイル設定などで使える）
}

# エッジの定義
graph_renderer.edge_renderer.data_source.data = {
    "start": ["A", "B"],  # ノードの始点リスト
    "end": ["B", "C"],  # ノードの終点リスト
}

# (2) ノード配置方法の定義
graph_layout = {"A": (0, 1), "B": (1, 0), "C": (-1, 0)}  # 各ノードのXY座標値が入った辞書
graph_renderer.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)

# (3) スタイルの設定
# ノードのスタイルの定義
graph_renderer.node_renderer.glyph = Circle(size=25, fill_color="my_color")

# エッジのスタイルの定義
graph_renderer.edge_renderer.glyph = MultiLine(
    line_color="#CCCCCC",  line_width=4
)

# ★ノードのラベル★
d = 0.05
label_source = ColumnDataSource(
    {"text": ["A", "B", "C"], "x": [0, 1, -1], "y": [1 + d, 0 + d, 0 + d]}
)
text_glyph = Text(x="x", y="y", text="text", text_color="#000000")
plot.add_glyph(label_source, text_glyph)

# プロットにネットワーク図を追加
plot.renderers.append(graph_renderer)

# ノートブック上に出力
output_notebook()
# ファイルに出力
# output_file("bokeh_graph.html")

# プロットを表示
show(plot)

Loading BokehJS ...

# BokehとNetworkXの連携

In [5]:
# NetworkXの空手データの確認
import networkx as nx

G = nx.karate_club_graph()
G.nodes(data=True)

NodeDataView({0: {'club': 'Mr. Hi'}, 1: {'club': 'Mr. Hi'}, 2: {'club': 'Mr. Hi'}, 3: {'club': 'Mr. Hi'}, 4: {'club': 'Mr. Hi'}, 5: {'club': 'Mr. Hi'}, 6: {'club': 'Mr. Hi'}, 7: {'club': 'Mr. Hi'}, 8: {'club': 'Mr. Hi'}, 9: {'club': 'Officer'}, 10: {'club': 'Mr. Hi'}, 11: {'club': 'Mr. Hi'}, 12: {'club': 'Mr. Hi'}, 13: {'club': 'Mr. Hi'}, 14: {'club': 'Officer'}, 15: {'club': 'Officer'}, 16: {'club': 'Mr. Hi'}, 17: {'club': 'Mr. Hi'}, 18: {'club': 'Officer'}, 19: {'club': 'Mr. Hi'}, 20: {'club': 'Officer'}, 21: {'club': 'Mr. Hi'}, 22: {'club': 'Officer'}, 23: {'club': 'Officer'}, 24: {'club': 'Officer'}, 25: {'club': 'Officer'}, 26: {'club': 'Officer'}, 27: {'club': 'Officer'}, 28: {'club': 'Officer'}, 29: {'club': 'Officer'}, 30: {'club': 'Officer'}, 31: {'club': 'Officer'}, 32: {'club': 'Officer'}, 33: {'club': 'Officer'}})

In [6]:
# bokehへの変換
from bokeh.plotting import from_networkx

# NetworkXを使って空手データの読み込み
G = nx.karate_club_graph()

# プロットを作成
plot = Plot(width=400, height=400)

# NetworkXのGraphオブジェクト ⇨ bokehのGraphRenderer
# ※ NetworkXのレイアウト関数を指定すると、変換時に位置を計算する
graph_renderer = from_networkx(G, nx.spring_layout, scale=1, center=(0, 0))

# スタイルを設定
graph_renderer.node_renderer.glyph = Circle(size=15, fill_color="blue")
graph_renderer.edge_renderer.glyph = MultiLine(
    line_color="gray", line_alpha=0.8, line_width=1
)
plot.renderers.append(graph_renderer)

# 表示
show(plot)

# インタラクティブな操作（移動/選択など）

In [7]:
from bokeh.models import HoverTool, NodesAndLinkedEdges

# プロットを作成
plot = figure(plot_width=400, plot_height=400)

# ネットワーク図のためのレンダラを作成
graph_renderer = GraphRenderer()

# (1) ネットワーク構造の定義
# ノードの定義
graph_renderer.node_renderer.data_source.data = {
    "index": ["A", "B", "C"],  # ノードID（必須）
    "name": ["Alice", "Bob", "Carol"],  # ★
    "club": ["科学部", "美術部", "ボクシング部"]  # ★
}

# エッジの定義
graph_renderer.edge_renderer.data_source.data = {
    "start": ["A", "B"],  # ノードの始点リスト
    "end": ["B", "C"],  # ノードの終点リスト
}

# スタイルの設定
# ノードのスタイルの定義
graph_renderer.node_renderer.glyph = Circle(size=25, fill_color="gray")
# エッジのスタイルの定義
graph_renderer.edge_renderer.glyph = MultiLine(
    line_color="#CCCCCC", line_alpha=0.8, line_width=2
)

# ▼▼▼▼▼▼▼▼▼▼▼▼
# インタラクティブな設定
# (1) ホバー時に表示する情報を指定
node_hover_tool = HoverTool(tooltips=[("index", "@index"), ("名前", "@name"), ("部活動", "@club")])
# ホバーツールなどのツールを追加
plot.add_tools(node_hover_tool)

# (2) ホバー時のスタイルのポリシーを指定
graph_renderer.inspection_policy = NodesAndLinkedEdges()

# ホバーした時のノード・エッジのスタイルを指定
graph_renderer.node_renderer.hover_glyph = Circle(size=25, fill_color="red")
graph_renderer.edge_renderer.hover_glyph = MultiLine(line_color="skyblue", line_width=5)
# ▲▲▲▲▲▲▲▲▲▲▲▲

# ノード配置方法の定義
graph_layout = {"A": (0, 1), "B": (1, 0), "C": (-1, 0)}  # 各ノードのXY座標値が入った辞書
graph_renderer.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)

# プロットにネットワーク図を追加
plot.renderers.append(graph_renderer)

# ノートブック上に出力
output_notebook()
# ファイルに出力
# output_file("bokeh_graph.html")

# プロットを表示
show(plot)

Loading BokehJS ...

# インタラクティブな操作（コールバック関数）

## ドロップダウンでノードの色を変更する

In [8]:
from bokeh.models import CustomJS, Dropdown
from bokeh.layouts import column, row

# プロットを作成
plot = figure(plot_width=400, plot_height=400)

# ネットワーク図のためのレンダラを作成
graph_renderer = GraphRenderer()

# (1) ネットワーク構造の定義
# ノードの定義
graph_renderer.node_renderer.data_source.data = {
    "index": ["A", "B", "C"],  # ノードID（必須）
    "name": ["Alice", "Bob", "Carol"],
    "club": ["科学部", "美術部", "ボクシング部"],
    "color": ["gray", "gray", "gray"]
}

# エッジの定義
graph_renderer.edge_renderer.data_source.data = {
    "start": ["A", "B"],  # ノードの始点リスト
    "end": ["B", "C"],  # ノードの終点リスト
}

# スタイルの設定
# ノードのスタイルの定義
graph_renderer.node_renderer.glyph = Circle(size=25, fill_color="color")
# エッジのスタイルの定義
graph_renderer.edge_renderer.glyph = MultiLine(
    line_color="#CCCCCC", line_alpha=0.8, line_width=2
)

# ▼▼▼▼▼▼▼▼▼▼▼▼
# 色を選択するためのドロップダウンを作成
menu = [("Red", "red"), ("Blue", "blue"), ("Gray", "gray")]
dropdown = Dropdown(label="Dropdown button", menu=menu)

# ドロップダウンで選択した時のコールバック関数をJavaScriptで定義
callback_dropdown_selected = CustomJS(args=dict(
        node_data_source=graph_renderer.node_renderer.data_source, dropdown=dropdown
    ),
    code="""
    console.log(this.item + 'をタップしました')
    console.log(node_data_source)
    var new_color = this.item
    node_data_source['data']["color"] = [new_color, new_color, new_color]
    node_data_source.change.emit()
""")
# ドロップダウンにコールバックを登録
dropdown.js_on_event(
    "menu_item_click",
    callback_dropdown_selected
)
# ▲▲▲▲▲▲▲▲▲▲▲▲

# ノード配置方法の定義
graph_layout = {"A": (0, 1), "B": (1, 0), "C": (-1, 0)}  # 各ノードのXY座標値が入った辞書
graph_renderer.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)

# プロットにネットワーク図を追加
plot.renderers.append(graph_renderer)

# ノートブック上に出力
output_notebook()
# ファイルに出力
# output_file("bokeh_interactive_graph.html")

# プロットを表示
show(column(dropdown, plot))

Loading BokehJS ...

## タップしたノードの情報をテキストエリアに表示する

In [9]:
from bokeh.models.callbacks import CustomJS
from bokeh.models import TextAreaInput, Div
from bokeh.models import TapTool

# プロットを作成
plot = figure(plot_width=400, plot_height=400)

# ネットワーク図のためのレンダラを作成
graph_renderer = GraphRenderer()

# (1) ネットワーク構造の定義
# ノードの定義
graph_renderer.node_renderer.data_source.data = {
    "index": ["A", "B", "C"],  # ノードID（必須）
    "name": ["Alice", "Bob", "Carol"],  # ★
    "club": ["科学部", "美術部", "ボクシング部"],  # ★
}

# エッジの定義
graph_renderer.edge_renderer.data_source.data = {
    "start": ["A", "B"],  # ノードの始点リスト
    "end": ["B", "C"],  # ノードの終点リスト
}

# スタイルの設定
# ノードのスタイルの定義
graph_renderer.node_renderer.glyph = Circle(size=25, fill_color="gray")
# エッジのスタイルの定義
graph_renderer.edge_renderer.glyph = MultiLine(
    line_color="#CCCCCC", line_alpha=0.8, line_width=2
)

# ▼▼▼▼▼▼▼▼▼▼▼▼
# タップしたノードの情報を表示するためのdiv領域を作成
info_div = Div(
    width=700, height=50, style={"background-color": "#CCCCCC", "font-size": "30px"}
)
info_div.text = "ここにタップしたノードの情報が表示されます"

# タップした時のコールバック関数をJavaScriptで定義
callback_tap = CustomJS(
    args=dict(
        node_data_source=graph_renderer.node_renderer.data_source, info_div=info_div
    ),
    code="""
    // タップしたノードのインデックスのリスト
    var selected_indices = node_data_source.selected['indices']
    console.log(selected_indices + 'をタップしました')

    // タップしたノードの情報を取得する
    var node_dict_list = []
    for (var i = 0; i < selected_indices.length; ++i){
        var selected_index = selected_indices[i]
        var node_dict = {}
        for (let key in node_data_source['data']) {
            var values = node_data_source['data'][key]
            node_dict[key] = values[selected_index]
        }
        node_dict_list.push(JSON.stringify(node_dict))
    }
    // div領域にノード情報を表示
    info_div.text = node_dict_list.join(',');
    // デバッグ用の出力
    console.log(node_dict)
""",
)
# コールバック関数を登録
plot.add_tools(TapTool(callback=callback_tap))
# ▲▲▲▲▲▲▲▲▲▲▲▲

# ノード配置方法の定義
graph_layout = {"A": (0, 1), "B": (1, 0), "C": (-1, 0)}  # 各ノードのXY座標値が入った辞書
graph_renderer.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)

# プロットにネットワーク図を追加
plot.renderers.append(graph_renderer)

# ノートブック上に出力
output_notebook()

# プロットを表示
show(column(info_div, plot))

Loading BokehJS ...